In [1]:
import speech_recognition as sr    # SESİ METİNE ÇEVİRMEK İÇİN
import numpy as np 
import pandas as pd                # DATA SETİNİ OKUMA VE DÜZENLEME

In [2]:
# RNN MODELİNİ KURMAK İÇİN KERAS KUTUPHANESİNİ KULLANACAĞIZ
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [3]:
import pandas as pd

# Dosyayı utf-16 kodlaması ile oku
dataset = pd.read_csv("C:/Users/bgurd/magaza_yorumlari_duygu_analizi.csv", encoding='utf-16')

etiket = dataset['Durum'].values.tolist()  
data = dataset['Görüş'].values.tolist()  


In [4]:
# Tüm verileri string'e dönüştür
data = [str(review) for review in data]

# Veriyi eğitim ve test setlerine ayırın
x_train, x_test, y_train, y_test = train_test_split(data, etiket, test_size=0.2, random_state=42)

In [5]:
# Tüm verileri string'e dönüştür
data = [str(review) for review in data]

# Tokenizer kullanarak metinleri tokenize et
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(data)


In [6]:
# Tüm verileri string'e dönüştür (emin olmak için)
x_train = [str(review) for review in x_train]
x_test = [str(review) for review in x_test]

# Tokenleştirme
x_train_tokens = tokenizer.texts_to_sequences(x_train)  # Eğitim setini tokenleştirdik
x_test_tokens = tokenizer.texts_to_sequences(x_test)    # Test setini tokenleştirdik

In [7]:
# Kontrol amaçlı ilk birkaç tokenleştirilmiş metni yazdır
print(x_train_tokens[:5])
print(x_test_tokens[:5])

[[26, 1060, 14, 3643, 1819, 753, 4304], [93, 2, 120, 3305, 6886, 584, 4147, 3306, 14, 14], [4, 479, 11, 5760, 13, 24], [16, 152, 173, 24], [16, 1, 4938, 66, 47, 703, 2, 42, 1, 4, 33, 2695, 4034, 80, 2, 7, 641, 172, 6335, 13, 24, 218, 13, 396, 131, 3, 108]]
[[2041, 4275, 2780, 605, 6293, 289, 588, 449, 2, 358, 1, 257, 20, 3253, 2886, 6633, 5589, 145, 320, 2093, 1, 39, 76, 69, 1326, 78, 114, 2164, 119, 3702, 1352, 5, 213, 13, 102], [2, 152, 8273, 504, 212, 783, 10, 57, 608, 25, 2, 35, 90, 38, 45, 123, 557, 52], [1, 2189, 2676, 104, 2, 7, 143, 2, 59], [275, 7459, 356, 912, 1238, 1007, 727, 381, 190, 79, 705, 4398, 2996, 20, 11, 202, 4945, 1870, 4805, 12], [35, 655, 3, 2200, 2660, 385, 4, 21, 7, 1, 4]]


In [8]:
## RNN aynı boyutta veri kümelerinde çalıştığından datalar kısaltcaz veya 0 atarak uzatcaz (eşitlemek için)
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens ]   ## her yorumda kac token oldugu hesaplanır
num_tokens = np.array(num_tokens) ## dizi numpy arraye ceviriyoruz
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens) ## cumlelerin ortalama kelime sayısının 2 fazlasıyla standart sapmasını topluyoruz
max_tokens = int(max_tokens)

In [9]:
# Pading eklenmiş verileri tanımlayın
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens) # pad_sequences keras kutupanesının fonksıyonudur
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)

# Daha sonra yeniden şekillendirme işlemini gerçekleştirin
x_train_pad_reshaped = x_train_pad.reshape(x_train_pad.shape[0], -1)
x_test_pad_reshaped = x_test_pad.reshape(x_test_pad.shape[0], -1)

In [10]:
model = Sequential() ## ardışık model oluşturma

In [11]:
embedding_size = 35 ## kullanılcak kelime vektörlerinin uzunluğu

In [12]:
# input içindeki kelimelerin vektörlerini output olarak verir
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer'))          ## 9000 e 50 boyutunda matris oluşturuyoruz

C:\Users\bgurd\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [13]:

model.add(GRU(units= 16, return_sequences=True)) # unit = 16 00 nöron sayısı // return_sequence bir sonraki layerları ekleyecegımız ıcın true
model.add(GRU(units= 8, return_sequences=True))
model.add(GRU(units= 4)) # son noron oldugu ıcın return_sequeces oto false bırakıyoruz
model.add(Dense(1, activation='sigmoid')) # cıkıs noronu oldugu ıcın Dense kullanıldı ve sigmoid fonk 0 ile 1 arası degerler doner yanı mutlu ve mutsuz

In [14]:
# Optimizasyon
optimizer = Adam(learning_rate=1e-3)  # 1e-3 = 0.001 

In [15]:
# Modelin Derlenmesi 
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy']) # yanlızca 2 sınıf oldugu ıcın loss için binary_crossentropy kullandık // metris masarı oranını gormek ıcın kullanıldı

In [16]:
print(y_train[:10])  # İlk 10 etiketi yazdır
print(y_test[:10])   # Test setindeki ilk 10 etiketi yazdır

['Tarafsız', 'Tarafsız', 'Olumlu', 'Olumlu', 'Olumlu', 'Olumlu', 'Olumsuz', 'Olumlu', 'Olumlu', 'Tarafsız']
['Olumsuz', 'Tarafsız', 'Olumlu', 'Olumsuz', 'Olumlu', 'Olumlu', 'Olumsuz', 'Olumsuz', 'Tarafsız', 'Olumlu']


In [17]:
label_to_index = {"Olumlu": 1, "Olumsuz": 0, "Tarafsız": 2}

y_train_index = np.array([label_to_index[label] for label in y_train])
y_test_index = np.array([label_to_index[label] for label in y_test])

In [18]:

## MODELİN EĞİTİMİ
model.fit(x_train_pad, y_train_index, epochs=20, batch_size=128)

Epoch 1/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 12s 60ms/step - accuracy: 0.3785 - loss: 0.5410
Epoch 2/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - accuracy: 0.3746 - loss: 0.3362
Epoch 3/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - accuracy: 0.5281 - loss: 0.0673
Epoch 4/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - accuracy: 0.6091 - loss: -0.2356
Epoch 5/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step - accuracy: 0.6577 - loss: -0.4147
Epoch 6/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 63ms/step - accuracy: 0.6680 - loss: -0.5904
Epoch 7/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 62ms/step - accuracy: 0.6893 - loss: -0.6640
Epoch 8/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 63ms/step - accuracy: 0.6940 - loss: -0.8486
Epoch 9/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 63ms/step - accuracy: 0.6930 - loss: -0.9334
Epoch 10/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 65ms/step - accuracy: 0.6990 - loss: -1.0554
Epoch 11/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 64ms/step - accuracy: 0.6990 - loss: -1.1819
Epoch 12/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 65ms/step - 

In [36]:
r = sr.Recognizer()
import emoji
facesmiling='\U0001F600'

with sr.Microphone() as source:
    print('speak : ')
    audio = r.listen(source)
    
    try:
        text = r.recognize_google(audio, language='tr-tr')
        print("soylediginiz : {}".format(text))
        texts = [text]
        tokens = tokenizer.texts_to_sequences(texts)
        tokens_pad = pad_sequences(tokens, maxlen= max_tokens)
        deger = model.predict(tokens_pad)
        print(deger)
        if deger > 0.5:
            print('Pozitif : \U0001F642')
        else:
            print('Negatif : \U0001F641') 
        
    except:
        print("anlamadım")

speak : 
soylediginiz : model düzgün çalışmıyor
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
[[0.02029321]]
Negatif : 🙁
